**MAKING PICTURES

In [1]:
from PIL import Image
import PIL
import numpy as np
import IPython.display
from random import randint, seed

from Bio import SeqIO
import math

fasta_file = "myco_genome.fasta"

# seq_file = "genome.png"
# mask_file = "genome_mask.png"


# import sys
# stdout = sys.stdout
# reload(sys)
# sys.setdefaultencoding('utf-8')
# sys.stdout = stdout

In [2]:
#READ SEQ
for seq_record in SeqIO.parse(fasta_file, "fasta"):
    seqarray = np.asarray(seq_record.seq) 
    

# n = 2 ** 14 //4
# blocks =  ["A"] * n +["G"] * n +["C"] * n +["T"] * n 
# seqarray = np.array(blocks) 

In [3]:
print(seqarray.shape)
print(seqarray)

(4399916,)
['T' 'T' 'G' ... 'T' 'C' 'G']


In [4]:
#SEQ

# length = len(seqarray)
length = 2 ** 14
print(length)

seq_np = np.zeros((4, length), dtype = np.float32)

channels = {"A": 0, "T": 1, "C": 2, "G": 3}


for index in range(length):
    base = seqarray[index]
    channel = channels[base]
    seq_np[channel][index] = 1
    


16384


In [5]:
print(seq_np.shape)
# print(seq_np)

(4, 16384)


In [6]:
#MASK
seed(7)
mask_np = np.zeros((4, length), dtype=np.float32)
mask_np.fill(1)

length_mask = 1000

for n in range(length_mask):
    spot = 1
    index = randint(0, length-spot)
    for i in range(index, index+spot):
        mask_np[:, i] = [0,0,0,0]


In [7]:
print(mask_np.shape)
# print(mask_np)

(4, 16384)


# Deep image prior 

## Import libs 

In [8]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
import torch
import torch.optim

from utils.inpainting_utils import *

# torch.backends.cudnn.enabled = True
# torch.backends.cudnn.benchmark =True
# dtype = torch.cuda.FloatTensor

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
dtype = torch.FloatTensor

# PLOT = True
PLOT = False
imsize = -1
# dim_div_by = 64
dim_div_by = 64

# Setup

In [9]:
pad = 'reflection' # 'zero'
OPT_OVER = 'net'
OPTIMIZER = 'adam'
NET_TYPE = 'skip_depth6'

In [10]:
if True:
    # Same params and net as in super-resolution and denoising
    INPUT = 'noise'
    input_depth = 32
    LR = 0.01 
    num_iter = 21
    param_noise = False
    show_every = 5
    figsize = 5 #????
    reg_noise_std = 0.03
    
#     net = skip(input_depth, img_np.shape[0], 
#                num_channels_down = [128] * 2,
#                num_channels_up =   [128] * 2,
#                num_channels_skip =    [128] * 2,  
#                filter_size_up = 3, filter_size_down = 3, 
#                upsample_mode='nearest', filter_skip_size=1,
#                need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)

    net = skip(input_depth, seq_np.shape[0], #change skip function in models/skip.py
               num_channels_down = [128] * 3,
               num_channels_up =   [128] * 3,
               num_channels_skip =    [128] * 3,  
               filter_size_up = 3, filter_size_down = 3, 
               upsample_mode='nearest', filter_skip_size=1,
               need_sigmoid=True, need_bias=True, pad=pad, act_fun='LeakyReLU').type(dtype)
    

net = net.type(dtype) 

net_input = get_noise(input_depth, INPUT, seq_np.shape[1]).type(dtype) #tensor 
# print(net_input) #tensor of noise
# print(net)# all net design

In [11]:
# Compute number of parameters
s  = sum(np.prod(list(p.size())) for p in net.parameters())
print ('Number of params: %d' % s)

# Loss
mse = torch.nn.MSELoss().type(dtype)

# img_var = np_to_torch(img_np).type(dtype)
# mask_var = np_to_torch(img_mask_np).type(dtype)

img_var = np_to_torch(seq_np).type(dtype)
mask_var = np_to_torch(mask_np).type(dtype)

print(img_var.shape)
print(mask_var.shape)

Number of params: 646788
torch.Size([1, 4, 16384])
torch.Size([1, 4, 16384])


# Main loop

In [12]:
# net(net_input)

In [13]:
from utils.inpainting_utils import *
i = 0
def closure():
    
    global i
    
#     if param_noise:
#         for n in [x for x in net.parameters() if len(x.size()) == 4]:
#             n = n + n.detach().clone().normal_() * n.std() / 50
    
#     net_input = net_input_saved
#     if reg_noise_std > 0:
#         net_input = net_input_saved + (noise.normal_() * reg_noise_std)
        
        
    out = net(net_input)
   
    total_loss = mse(out * mask_var, img_var * mask_var)
#     print(total_loss) #tensor
    total_loss.backward()
        
    print ('Iteration %05d    Loss %f' % (i, total_loss.item()), '\r', end='')
#     if  PLOT and i % show_every == 0:
#         out_np = torch_to_np(out)
#         plot_image_grid([np.clip(out_np, 0, 1)], factor=figsize, nrow=1)
        
    i += 1

    return total_loss


net_input_saved = net_input.detach().clone()
# print(net_input_saved.shape)
noise = net_input.detach().clone()

p = get_params(OPT_OVER, net, net_input) # list of tensors to optimize over !! in optimize

optimize(OPTIMIZER, p, closure, LR, num_iter) # optimize is in utils/common.utils


Starting optimization with ADAM


/Users/pochtalionizm/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [14]:
out_np = torch_to_np(net(net_input))
# plot_image_grid([out_np], factor=5);

In [15]:
print(out_np.shape)

(4, 16384)


In [16]:
# channels = {"A": 0, "T": 1, "C": 2, "G": 3}


# bases = {"A": np.array([1, 0, 0, 0], dtype = np.float32),
#          "T": np.array([0, 1, 0, 0], dtype = np.float32),
#          "C": np.array([0, 0, 1, 0], dtype = np.float32),
#          "G": np.array([0, 0, 0, 1], dtype = np.float32)
#         }

# np.linalg.norm(bases["A"] - bases["G"])

In [17]:
channels = ["A", "T", "C", "G"]
out_seq = np.zeros(length, dtype= "U8")
for i in range(length):
    channel = np.argmax(out_np[:, i])
    out_seq[i] = channels[channel]   

In [18]:
out_array = np.zeros((4,length))
for i in range(length):
    n = np.argmax(out_np[:, i])
    out_array[n, i] = 1

In [19]:
diff = np.zeros(length)
# print(type(diff[0]))
print(np.array_equal(out_array[:, i], seq_np[:, i]))

counter = {"all": 0, "mask": 0, "free": 0}

for i in range(length):
    if not np.array_equal(out_array[:, i], seq_np[:, i]):
        diff[i] = 1
        counter["all"] +=1
        if mask_np[1, i] == 0:
            counter["mask"] += 1
        else:
            counter["free"] += 1
        
print("mask\t{}, free\t{}".format(counter['mask']/length_mask, counter["free"]/(length - length_mask)))

True
mask	0.72, free	0.12057982319292772
